## Import

In [ ]:
import numpy as np
import json
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import fastai
from fastai import *
from fastai.data.core import DataLoaders
from fastai.learner import Learner
from fastai.losses import CrossEntropyLossFlat
from fastai.metrics import accuracy
from fastai.optimizer import Adam
from fastai.callback.progress import ProgressCallback
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from parse_preprocessed_data import get_inputs_and_targets
from LSTM_Model import LSTMModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Hyper-Parameters

In [ ]:
seq_length = 2000

hidden_size = 512
learning_rate = 2e-3
dropout = 0
batch_size = 100
num_layers = 3
max_epochs = 15
validation_prop = 0.2

## Load Data

In [ ]:
#Not Original
char_to_ix, ix_to_char, vocab_size, inputs, targets = get_inputs_and_targets('data_preprocessed/mario.txt', seq_length)
vocab_size, inputs.shape, targets.shape

In [ ]:
#Not Original
first_three_cols = inputs[0][:3 * 17]
np.savetxt('data_preprocessed/seed.txt', first_three_cols)

In [ ]:
#Not Original
with open('data_preprocessed/char_to_ix.json', 'w+') as json_f:
    json.dump(char_to_ix, json_f)

with open('data_preprocessed/ix_to_char.json', 'w+') as json_f:
    json.dump(ix_to_char, json_f)

In [ ]:
inputs.shape,inputs.dtype

In [ ]:
targets.shape, targets.dtype

In [ ]:
class SequenceDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

train_inputs, valid_inputs, train_targets, valid_targets = train_test_split(inputs, targets, test_size=validation_prop)

# Define your training and validation datasets and DataLoaders
train_dataset = SequenceDataset(train_inputs, train_targets)
valid_dataset = SequenceDataset(valid_inputs, valid_targets)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)

# Combine your training and validation DataLoaders into a DataLoaders object
dls = DataLoaders(train_dataloader, valid_dataloader)

## Model Callbacks

In [ ]:
def custom_loss(y_pred_tup, y_true):
    y_pred,_,_,_,_,_,_ = y_pred_tup
    return F.cross_entropy(
        y_pred.view(-1, vocab_size),
        y_true.long().view(-1)
    )

In [ ]:
def custom_acc(y_pred_tup, y_true):
    y_pred,_,_,_,_,_,_ = y_pred_tup
    pred_classes = torch.argmax(y_pred.view(-1, vocab_size), dim=1)
    true_classes = y_true.view(-1)
    class_equality = torch.eq(pred_classes, true_classes).float()
    return torch.mean(class_equality)

## Model

In [ ]:
# Initialize model
model = LSTMModel(vocab_size=vocab_size, hidden_size=hidden_size, dropout=dropout).to(device)

In [ ]:
learn = Learner(dls, model, opt_func =Adam, loss_func=custom_loss, metrics=custom_acc)

## Train Model

In [ ]:
learn.fit(n_epoch=max_epochs, lr = learning_rate)

In [ ]:
#Not Original
seed = np.loadtxt('data_preprocessed/seed.txt', dtype=float)[:3*17 - 1].copy()

with open('data_preprocessed/ix_to_char.json', 'r') as json_f:
    ix_to_char = json.load(json_f)
    
with open('data_preprocessed/char_to_ix.json', 'r') as json_f:
    char_to_ix = json.load(json_f)

In [ ]:
#Not Original
def onehot_to_string(onehot):
    ints = np.argmax(onehot, axis=-1)
    chars = [ix_to_char[str(ix)] for ix in ints]
    string = "".join(chars)
    char_array = []
    for line in string.rstrip().split('\n')[:-1]:
        if len(line) == 16:
            char_array.append(list(line))
        elif len(line) > 16:
            char_array.append(list(line[:16]))
        elif len(line) < 16:
            char_array.append(['-'] * (16 - len(line)) + list(line))
    char_array = np.array(char_array).T
    string = ""
    for row in char_array:
        string += "".join(row) + "\n"
    return string

In [ ]:
#Not Original
seed[17+14] = 0
seed[17+14][char_to_ix['x']] = 1
seed[17*2+14] = 0
seed[17*2+14][char_to_ix['x']] = 1
print(onehot_to_string(seed))

In [ ]:
#Not Original
def get_seed():
    seed = np.loadtxt('data_preprocessed/seed.txt', dtype=float)[:3*17 - 1]
    seed[17+14] = 0
    seed[17+14][char_to_ix['x']] = 1
    seed[17*2+14] = 0
    seed[17*2+14][char_to_ix['x']] = 1
    return seed

In [ ]:
#Not Original
seed = get_seed()
seed.shape

In [ ]:
#Not Original
num_levels_to_gen = 10

num_chunks = 10
num_cols_per_chunk = 16
num_rows_per_col = 17
num_chars_to_gen = num_chunks * num_cols_per_chunk * num_rows_per_col - len(seed)
print(num_chars_to_gen)

In [ ]:
# Prepare seed
seed = get_seed()
seed = np.expand_dims(seed, axis=0)
seed = np.repeat(seed, num_levels_to_gen, axis=0)

gen = seed.copy()

# Initialize all hidden and cell states to zeros
lstm1_h = torch.zeros((1, num_levels_to_gen, hidden_size)).to(device)
lstm1_c = torch.zeros((1, num_levels_to_gen, hidden_size)).to(device)
lstm2_h = torch.zeros((1, num_levels_to_gen, hidden_size)).to(device)
lstm2_c = torch.zeros((1, num_levels_to_gen, hidden_size)).to(device)
lstm3_h = torch.zeros((1, num_levels_to_gen, hidden_size)).to(device)
lstm3_c = torch.zeros((1, num_levels_to_gen, hidden_size)).to(device)
# Generate characters
for i in tqdm(range(num_chars_to_gen), leave=False):
    # Convert seed to tensor and move to device
    seed_tensor = torch.tensor(seed, dtype=torch.float32).to(device)
    
    # Predict probas and update hidden and cell states using the trained model
    probas, lstm1_h, lstm1_c, lstm2_h, lstm2_c, lstm3_h, lstm3_c = learn.model(
        seed_tensor, lstm1_h, lstm1_c, lstm2_h, lstm2_c, lstm3_h, lstm3_c
    )

    probas = probas[:, -1]  # All batches, last timestep
    
    seed = np.zeros((num_levels_to_gen, 1, vocab_size))
    for b in range(num_levels_to_gen):
        p = probas[b]
        normalized_probas = probas[b] / torch.sum(probas[b])
        idx = np.random.choice(np.arange(len(p)), p=normalized_probas.cpu().detach().numpy())
        seed[b][0] = 0
        seed[b][0][idx] = 1
        
    # Concatenate generated characters
    gen = np.concatenate([gen, seed], axis=1)


In [ ]:
#Not Original
gen.shape

In [ ]:
#Not Original
for i, g in enumerate(gen):
    with open(f'generated_levels_txt/{i+1}.txt', 'w+') as txt_f:
        txt_f.write(onehot_to_string(g))